### Chapter 9: 2D axissymmetric tri element

Isoparametric formulation

In [12]:
import numpy as np
import sympy as sp
sp.init_printing()

### The 2D axissymmetric tri element

This is a triangular element with nodes: i, j, m.

Coordinates are: r, z for radial and vertical directions, respectively.

Each node hat 2 degrees of freedom (DOFs):
- u: radial displacement
- w: vertical displacement

The DOFs are ordered as follows: u, w.

### The physical properties of the element

In [13]:

### The physical properties of the element
E = 2.0e5  # Young's modulus in Pa
nu = 0.25  # Poisson's ratio
t = 1000  # thickness in mm
t /= 1000  # convert to m

# Define symbolic physical coordinates
r1, z1, r2, z2, r3, z3 = sp.symbols('r1 z1 r2 z2 r3 z3')

# Example coordinates for the nodes
# physical_coords = {'P1': (0, 0),
#                    'P2': (1, 0),
#                    'P3': (0, 1)}  # example coordinates for the nodes
physical_coords = {'P1': (4, 0),
                   'P2': (6, 2),
                   'P3': (3, 1)}  # example coordinates for the nodes
physical_coords = {'P1': (0, 0),
                   'P2': (3, 0),
                   'P3': (0, 2)}  # example coordinates for the nodes

midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
            (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3

physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
                       'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
                       'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1]}

R, Z = sp.symbols('R Z')  # radial and vertical coordinates

# Define the physical nodes as a matrix so it can be used a symbolic evaluation
physical_nodes = sp.Matrix([[r1, z1],
                            [r2, z2],
                            [r3, z3]])

In [14]:
ND = 2  # number of degrees of freedom per node
NNODE = 3  # number of nodes

### The isoparametric part

In [15]:
# coordinates of the nodes in the element
n1 = np.array([0, 0])  # node 1 coordinates
n2 = np.array([1, 0])   # node 2 coordinates
n3 = np.array([0, 1])    # node 3 coordinates
natural_nodes = np.array([n1, n2, n3])  # all nodes

### Shape functions and their derivatives

In [16]:
# Define symbolic variables. Capital letters S, T are the symbols for the variables s, t
# S, T = sp.symbols('s t')

# Define the shape functions symbolically
def shape_functions() -> sp.Matrix:
    """shape functions"""

    return sp.Matrix([
        1 - R - Z,
        R,
        Z,
    ])

def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 1])])

# Calculate the derivatives by s and t, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/ds, [1] = dNi/dt
    """
    sh = shape_functions()  # N1, N2, N3, N4
    dN_ds = []
    dN_dt = []

    for i in range(len(sh)):
        dN_ds.append(sp.diff(sh[i], R))
        dN_dt.append(sp.diff(sh[i], Z))

    return dN_ds, dN_dt

print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i+1}/ds = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i+1}/dt = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

# Substitute the symbolic coordinates with numerical values
st_values = {
    R: 0.3333,  # example value for s
    Z: 0.3333,  # example value for t
}
print()
print('The isoparametric mapping with coordinate values substituted is:')
x_map_substituted = x_map.subs(st_values)
y_map_substituted = y_map.subs(st_values)
print(f'x = {x_map_substituted}')
print(f'y = {y_map_substituted}')

print()
print('The derivatives of the shape functions by s are:')
print(derivatives[0])  # derivatives by s

print()
print('The derivatives of the shape functions by t are:')
print(derivatives[1])  # derivatives by t



The shape functions are:
Matrix([[-R - Z + 1], [R], [Z]])

The derivatives are:
dN1/ds = -1
dN2/ds = 1
dN3/ds = 0
dN1/dt = -1
dN2/dt = 0
dN3/dt = 1

The isoparametric mapping is:
x = R*r2 + Z*r3 + r1*(-R - Z + 1)
y = R*z2 + Z*z3 + z1*(-R - Z + 1)

The isoparametric mapping with coordinate values substituted is:
x = 0.3334*r1 + 0.3333*r2 + 0.3333*r3
y = 0.3334*z1 + 0.3333*z2 + 0.3333*z3

The derivatives of the shape functions by s are:
[-1, 1, 0]

The derivatives of the shape functions by t are:
[-1, 0, 1]


### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & N_j & 0 & N_m & 0 \\
0 & N_i & 0 & N_j & 0 & N_m\\
\end{bmatrix}
$$

In [17]:
def N() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[Ni, 0, Nj, 0, Nm, 0],
     [0, Ni, 0, Nj, 0, Nm]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

_N = N()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
            (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3
_N_e_node1 = _N_e.subs({'R': midpoint[0], 'Z': midpoint[1]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))




The shape function matrix N is (2, 6):

The shape function matrix N for this element:
⎡-R - Z + 1      0       R  0  Z  0⎤
⎢                                  ⎥
⎣    0       -R - Z + 1  0  R  0  Z⎦

The shape function matrix N for this element at node 1:
⎡-0.6667     0     1.0   0   0.6667    0   ⎤
⎢                                          ⎥
⎣   0     -0.6667   0   1.0    0     0.6667⎦


### Jacobian matrix

In [18]:
def jacobian() -> sp.Matrix:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # Calculate the derivatives
    dx_dr = sp.diff(x_map, R)
    dy_dr = sp.diff(y_map, R)
    dx_dz = sp.diff(x_map, Z)
    dy_dz = sp.diff(y_map, Z)

    return sp.Matrix([[dx_dr, dy_dr],
                      [dx_dz, dy_dz]])

# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_substituted = J.subs(physical_coords_map)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_substituted)
print()

# Evaluate the jacobian matrix at some points
for r_eval in [0, 0.3333]:
    for z_eval in [0, 0.3333]:
        J_evaluated = J_substituted.subs({R: r_eval, Z: z_eval})
        print(f'Jacobian matrix at r={r_eval}, z={z_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()

The jacobian matrix with coordinate values substituted is:
⎡3  0⎤
⎢    ⎥
⎣0  2⎦

Jacobian matrix at r=0, z=0:
⎡3  0⎤
⎢    ⎥
⎣0  2⎦
The determinant at this point is |J|=6.00

Jacobian matrix at r=0, z=0.3333:
⎡3  0⎤
⎢    ⎥
⎣0  2⎦
The determinant at this point is |J|=6.00

Jacobian matrix at r=0.3333, z=0:
⎡3  0⎤
⎢    ⎥
⎣0  2⎦
The determinant at this point is |J|=6.00

Jacobian matrix at r=0.3333, z=0.3333:
⎡3  0⎤
⎢    ⎥
⎣0  2⎦
The determinant at this point is |J|=6.00



### $\begin{bmatrix}B\end{bmatrix}$ matrix

$$\begin{bmatrix}B_i\end{bmatrix} = \begin{bmatrix}
\frac{\partial N_i}{\partial r} & 0 \\
0 & \frac{\partial N_i}{\partial z} \\
\frac{\partial N_i}{\partial z} & \frac{\partial N_i}{\partial r} \\
\frac{N_i}{r} & 0
\end{bmatrix}$$

One block of the B matrix, which relates the strain to the displacement for node i.

In [33]:
def calculate_B():
    """
    Calculates the complete B-matrix for the element.
    B relates the strain vector [ε_r, ε_z, γ_rz, ε_θ] to the nodal displacement vector.
    """
    J_inv = jacobian().inv()
    dN_dRZ = sp.Matrix(shape_function_derivatives()) # Matrix of shape function derivatives w.r.t R and Z

    # This calculates [dN/dr, dN/dz] for all N at once
    dN_drz = J_inv * dN_dRZ

    # The radius 'r' in the natural coordinate system
    r_iso = isoparametric_mapping()[0]

    N_funcs = shape_functions()
    B = sp.zeros(4, 6)

    for i in range(NNODE):
        # Derivatives for node i
        dNi_dr = dN_drz[0, i]
        dNi_dz = dN_drz[1, i]
        Ni = N_funcs[i]

        # Populate the B matrix, column by column
        # Node i contributes to columns 2*i and 2*i+1
        B[0, 2*i] = dNi_dr      # ε_r contribution from u_i
        B[1, 2*i+1] = dNi_dz    # ε_z contribution from w_i
        B[2, 2*i] = dNi_dz      # γ_rz contribution from u_i
        B[2, 2*i+1] = dNi_dr    # γ_rz contribution from w_i
        B[3, 2*i] = Ni / r_iso  # ε_θ (hoop strain) contribution from u_i

    return B

# Example of calculating and printing the B matrix
B_matrix = calculate_B()
# You can then substitute values as needed
B_evaluated = B_matrix.subs(physical_coords_map).subs({R: 1/3, Z: 1/3})
sp.pprint(B_evaluated.evalf().applyfunc(lambda x: round(x, 4)))

⎡-0.3333     0     0.3333    0       0      0 ⎤
⎢                                             ⎥
⎢   0      -0.5      0       0       0     0.5⎥
⎢                                             ⎥
⎢ -0.5    -0.3333    0     0.3333   0.5     0 ⎥
⎢                                             ⎥
⎣0.3333      0     0.3333    0     0.3333   0 ⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

In [20]:
def D_matrix():
    """
    The constitutive matrix for the axissymmetric tri element.
    """
    # Plane stress condition
    D = sp.zeros(4, 4)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu
    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu
    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu
    D[3, 3] = (1 - 2 * nu) / 2

    return D

### Quadrature points and weights

In [21]:
gi_data = {
    1: {
        'point': ((1/3, 1/3),),  # midpoint of the triangle
        'weight': (1,)
    },
    3: {
        'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
        'weight': (1/3, 1/3, 1/3)
    },
}

### The stiffness matrix $\begin{bmatrix}K\end{bmatrix}$

In [34]:
NG = 3

def K_matrix() -> np.array:
    """
    Gets the stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    J = jacobian()
    J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    print('The constitutive matrix D is:')
    sp.pprint(D)

    for i in range(NG):

        for p, w in zip(gi_data[NG]['point'], gi_data[NG]['weight']):
            print(f'Gauss point {i+1}: r={p[0]}, z={p[1]}, weight={w}')
            r = p[0]
            z = p[1]
            w = w

            # Calculate the jacobian at the gauss point
            J_evaluated = J_substituted.subs({R: r, Z: z}).evalf()
            det_J = J_evaluated.det()
            print('|J| at the Gauss point:')
            print(det_J)
            B_element = B_matrix.subs(physical_coords_map).subs({R: r, Z: z})
            # B_element = B_symbolic.subs(physical_coords_map)
            B = B_element.subs({R: r, Z: z}).evalf()
            print('B-Matrix at the Gauss point:')
            sp.pprint(B.applyfunc(lambda x: round(x, 4)))

            # Calculate the stiffness matrix contribution
            K_contribution = B.T * D * B * det_J * w
            K_lok += K_contribution
            # print()
            # print('K-Matrix contribution at {}, {}:'.format(r, z))
            # K_con_formatted = K_contribution.applyfunc(lambda x: x.round(0))
            # sp.pprint(K_con_formatted)

    return K_lok

# the full K matrix
print()
K_e = K_matrix()
print('The full stiffness matrix K is:')
sp.pprint(K_e.applyfunc(lambda x: round(x, 4)))


The constitutive matrix D is:
⎡0.75  0.25  0.25   0  ⎤
⎢                      ⎥
⎢0.25  0.75  0.25   0  ⎥
⎢                      ⎥
⎢0.25  0.25  0.75   0  ⎥
⎢                      ⎥
⎣ 0     0     0    0.25⎦
Gauss point 1: r=0.16666666666666666, z=0.16666666666666666, weight=0.3333333333333333
|J| at the Gauss point:
6.00000000000000
B-Matrix at the Gauss point:
⎡-0.3333     0     0.3333    0       0      0 ⎤
⎢                                             ⎥
⎢   0      -0.5      0       0       0     0.5⎥
⎢                                             ⎥
⎢ -0.5    -0.3333    0     0.3333   0.5     0 ⎥
⎢                                             ⎥
⎣1.3333      0     0.3333    0     0.3333   0 ⎦
Gauss point 1: r=0.6666666666666666, z=0.16666666666666666, weight=0.3333333333333333
|J| at the Gauss point:
6.00000000000000
B-Matrix at the Gauss point:
⎡-0.3333     0     0.3333    0       0      0 ⎤
⎢                                             ⎥
⎢   0      -0.5      0       0       0     0.5⎥
⎢

In [31]:
# the stiffness matrix as a numpy array
K_np = np.array(K_e).astype(np.float64)

# the eigenvalues
eigenvalues, eigenvectors = np.linalg.eigh(K_np)
print()
print('The eigenvalues of the stiffness matrix K are:')
eigenvalues = sorted([float(x) for x in eigenvalues])

print([round(ev, 5) for ev in eigenvalues])
# print(eigenvectors.shape)
# sp.pprint(eigenvectors)

# the eigenvectors for the near-zero eigenvalues
nearzero_eigenvalues = [ev for ev in eigenvalues if abs(ev) < 1e-10]
# printing the eigenvectors for the near-zero eigenvalues
print('The eigenvectors for the near-zero eigenvalues are:')
for i, ev in enumerate(nearzero_eigenvalues):
    print(f'Eigenvalue {i + 1}: {ev:.6f}')
    print('Eigenvector:')
    print(eigenvectors[:, i])
    print()


The eigenvalues of the stiffness matrix K are:
[-0.0, 0.3007, 1.55268, 4.3656, 4.79602, 17.67251]
The eigenvectors for the near-zero eigenvalues are:
Eigenvalue 1: -0.000000
Eigenvector:
[-8.07413591e-18  5.77350269e-01 -6.93889390e-17  5.77350269e-01
  2.49800181e-16  5.77350269e-01]

